In [1]:
# from fastchat.train.llama_flash_attn_monkey_patch import (
#     replace_llama_attn_with_flash_attn,
# )
# replace_llama_attn_with_flash_attn()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/jimx/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.8/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/jimx/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/jimx/miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/jimx/miniconda3 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/jimx/miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/home/jimx/.mujoco/mujoco210/bin')}
  warn(msg)


In [1]:
from transformers import AutoTokenizer, LlamaForCausalLM, GenerationConfig
import time
import torch

model_path = "/home/jimx/pretrained_models/meta-llama/Llama-2-13b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_path)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/jimx/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.8/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/jimx/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/jimx/miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/jimx/miniconda3 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/jimx/miniconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/home/jimx/.mujoco/mujoco210/bin')}
  warn(msg)


In [2]:
# add loading time:
start_time = time.time()
model = LlamaForCausalLM.from_pretrained(model_path,local_files_only=True,device_map="auto",use_safetensors=True,torch_dtype=torch.float16)
end_time = time.time()
print("Loading time: ", end_time-start_time)

# use_safetensors 13b:14.44
# no use_safetensors:

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

FileNotFoundError: No such file or directory: "/home/jimx/pretrained_models/meta-llama/Llama-2-13b-hf/model-00001-of-00003.safetensors"

In [9]:
generation_config = GenerationConfig(
    # temperature=0.7,
    # top_p=0.75,
    num_beams=5,
    do_sample=True
)

def evaluate(text):
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    start_time = time.time()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    end_time = time.time()
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    print(output)
    token_len = tokenizer.tokenize(output)
    # print token/second:
    print("token/second:",len(token_len) / (end_time - start_time))

In [ ]:
evaluate("what is the weather in New York?")

In [5]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [6]:
inp="""A Russian naval ship has been damaged in a Ukrainian naval drone attack in the Black Sea, Ukrainian sources say. The assault reportedly occurred near the Russian port of Novorossiysk, which is a major hub for Russian exports. Russia's defence ministry said it had repelled a Ukrainian attack on its naval base there which involved two sea drones, but did not admit any damage. But Ukrainian security service sources say the Olenegorsky Gornyak was hit and suffered a serious breach. They told the BBC a sea drone was carrying 450kg (992lb) of dynamite when it hit the ship. Russia made no mention of any damage in its report of the incident. Sea drones are small, unmanned vessels which operate on or below the water's surface. A video sent to the BBC by a source with Ukraine's security service appears to show the drone approaching a ship thought to be the Olenegorsky Gornyak. The footage shows a vessel travelling right up to the side of a ship before the feed cuts out, apparently on impact. Another unverified video is thought to show the ship listing to one side."""
x = generate_prompt("Please summarize the following text", inp)

In [7]:
evaluate(x)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please summarize the following text

### Input:
A Russian naval ship has been damaged in a Ukrainian naval drone attack in the Black Sea, Ukrainian sources say. The assault reportedly occurred near the Russian port of Novorossiysk, which is a major hub for Russian exports. Russia's defence ministry said it had repelled a Ukrainian attack on its naval base there which involved two sea drones, but did not admit any damage. But Ukrainian security service sources say the Olenegorsky Gornyak was hit and suffered a serious breach. They told the BBC a sea drone was carrying 450kg (992lb) of dynamite when it hit the ship. Russia made no mention of any damage in its report of the incident. Sea drones are small, unmanned vessels which operate on or below the water's surface. A video sent to the BBC by a source with Ukr